# S&P 500 Signal Classifier

Approach:
1. Create features
2. Winsorize data
3. Test, Train split data
4. Scale the test and train data
6. Original, Oversample, and Undersample the data
7. Apply RandomForestClassifier, XGBoost, and LightBoost to predict

## Install Required Packages

In [ ]:
# Install Yahoo Finance
!pip install yfinance --upgrade --no-cache-dir

In [ ]:
# Install SciKit Learn
!pip install scikit-learn

In [ ]:
# Install XGBoost
!pip install xgboost

In [ ]:
# Install LightBoost
!pip install lightgbm

## Import Required Dependencies

In [1]:
# Import the required libraries and dependencies
import yfinance as yf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score, make_scorer
from sklearn.model_selection import train_test_split
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from xgboost import XGBClassifier
from xgboost import plot_importance
import lightgbm as lgb

In [ ]:
import warnings
# Ignore warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

## Retrieve and Load Data

In [ ]:
# Retreive data
sp500 = yf.Ticker('^GSPC')
sp500 = sp500.history(period='max')

In [ ]:
# Drop timezone from datetime
sp500 = sp500.reset_index()
sp500['Date'] = sp500['Date'].dt.tz_localize(None)
sp500.set_index('Date', inplace=True)

In [ ]:
# Get the last rows of the DataFrame
sp500 = sp500.tail(24252) # Get max available data

In [ ]:
# Display data
sp500

In [ ]:
# Create stock df
stock_df = pd.DataFrame(sp500).dropna()
stock_df

In [ ]:
# Drop columns
stock_df.drop(columns={'Volume', 'Dividends', 'Stock Splits'}, inplace=True)

# Sort by ascending date
stock_df = stock_df.sort_values(by="Date", ascending=True)

# Review the first and last five rows of the DataFrame
display(stock_df.head())
display(stock_df.tail())

In [ ]:
# Rename dataframe to data
data = stock_df

## Perform Feature Engineering

In [ ]:
# Feature Engineering
# Moving Averages
data['SMA10'] = data['Close'].rolling(window=10).mean()
data['SMA20'] = data['Close'].rolling(window=20).mean()
data['SMA30'] = data['Close'].rolling(window=20).mean()
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['SMA100'] = data['Close'].rolling(window=100).mean()
data['SMA200'] = data['Close'].rolling(window=200).mean()

# RSI
def rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

data['RSI'] = rsi(data)

# MACD
data['EMA12'] = data['Close'].ewm(span=12, adjust=False).mean()
data['EMA26'] = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = data['EMA12'] - data['EMA26']
data['SL'] = data['MACD'].ewm(span=9, adjust=False).mean()

# Bollinger Bands
data['BollM'] = data['Close'].rolling(window=20).mean()
data['BollU'] = data['BollM'] + (2 * data['Close'].rolling(window=20).std())
data['BollL'] = data['BollM'] - (2 * data['Close'].rolling(window=20).std())

# Identify crossover signal for Target Column
# Close < Bollinger Mid and RSI < 50 and MACD < 0
data['Signal'] = np.where((data['Close'] < data['BollM']) & (data['RSI'] < 50) & (data['MACD'] < 0), 1, 0)

# Calculate return
data['Return'] = data['Close'].pct_change().shift(-1)  # Next day's return

# Label Data
# Buy ONLY if all Crossover conditions are met
# Assume profitable if return > 0%
data['Target'] = np.where((data['Signal'] > 0) & (data['Return'] > 0.0), 1, 0)

# Prepare dataset
features = ['SMA10', 'SMA20', 'SMA30', 'SMA50', 'SMA100', 'SMA200', 'Signal', 'RSI', 'MACD', 'SL', 'BollM', 'BollU', 'BollL']
dataset = data.dropna()[features + ['Target']]

## Perform Feature Engineering

In [ ]:
# Feature Engineering
# Moving Averages
data['SMA10'] = data['Close'].rolling(window=10).mean()
data['SMA20'] = data['Close'].rolling(window=20).mean()
data['SMA30'] = data['Close'].rolling(window=20).mean()
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['SMA100'] = data['Close'].rolling(window=100).mean()
data['SMA200'] = data['Close'].rolling(window=200).mean()

# RSI
def rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

data['RSI'] = rsi(data)

# MACD
data['EMA12'] = data['Close'].ewm(span=12, adjust=False).mean()
data['EMA26'] = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = data['EMA12'] - data['EMA26']
data['SL'] = data['MACD'].ewm(span=9, adjust=False).mean()

# Bollinger Bands
data['BollM'] = data['Close'].rolling(window=20).mean()
data['BollU'] = data['BollM'] + (2 * data['Close'].rolling(window=20).std())
data['BollL'] = data['BollM'] - (2 * data['Close'].rolling(window=20).std())

# Identify crossover signal for Target Column
# Close < Bollinger Mid and RSI < 50 and MACD < 0
data['Signal'] = np.where((data['Close'] < data['BollM']) & (data['RSI'] < 50) & (data['MACD'] < 0), 1, 0)

# Calculate return
data['Return'] = data['Close'].pct_change().shift(-1)  # Next day's return

# Label Data
# Buy ONLY if all Crossover conditions are met
# Assume profitable if return > 0%
data['Target'] = np.where((data['Signal'] > 0) & (data['Return'] > 0.0), 1, 0)

# Prepare dataset
features = ['SMA10', 'SMA20', 'SMA30', 'SMA50', 'SMA100', 'SMA200', 'Signal', 'RSI', 'MACD', 'SL', 'BollM', 'BollU', 'BollL']
dataset = data.dropna()[features + ['Target']]

## Perform Winsorization of Data

In [ ]:
from scipy.stats.mstats import winsorize
data['Open'] = winsorize(np.array(data['Open']), limits=[.05, .05])
data['High'] = winsorize(np.array(data['High']), limits=[.05, .05])
data['Low'] = winsorize(np.array(data['Low']), limits=[.05, .05])
data['Close'] = winsorize(np.array(data['Close']), limits=[.05, .05])
data['SMA10'] = winsorize(np.array(data['SMA10']), limits=[.05, .05])
data['SMA20'] = winsorize(np.array(data['SMA20']), limits=[.05, .05])
data['SMA30'] = winsorize(np.array(data['SMA30']), limits=[.05, .05])
data['SMA50'] = winsorize(np.array(data['SMA50']), limits=[.05, .05])
data['SMA100'] = winsorize(np.array(data['SMA100']), limits=[.05, .05])
data['SMA200'] = winsorize(np.array(data['SMA200']), limits=[.05, .05])
data['EMA12'] = winsorize(np.array(data['EMA12']), limits=[.05, .05])
data['EMA26'] = winsorize(np.array(data['EMA26']), limits=[.05, .05])
data['MACD'] = winsorize(np.array(data['MACD']), limits=[.05, .05])
data['BollM'] = winsorize(np.array(data['BollM']), limits=[.05, .05])
data['BollU'] = winsorize(np.array(data['BollU']), limits=[.05, .05])
data['BollL'] = winsorize(np.array(data['BollL']), limits=[.05, .05])
data['SL'] = winsorize(np.array(data['SL']), limits=[.05, .05])

In [ ]:
plt.figure(figsize=(20, 6))
sns.boxplot(data=data)
plt.title("Boxplot for Outlier Detection - Train")
plt.show()

## Perform Train and Test Splits

In [ ]:
# Define train and test sets
X = dataset[features]
y = dataset['Target']

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

## Perform Scaling

In [ ]:
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert scaled arrays back to DataFrames to maintain column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=features, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=features, index=X_test.index)

# Perform Classifications

## Original Scaled Data with Untuned RandomForestClassifier

In [ ]:
# Define the model
clf = RandomForestClassifier(random_state=1)

# Fit the model
clf.fit(X_train_scaled, y_train)

# Predict using the model
train_pred = clf.predict(X_train_scaled)
test_pred = clf.predict(X_test_scaled)

# Generate classification report
# Print scores
print("Balanced Accuracy Scores")
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training')
print(balanced_accuracy_score(y_test, test_pred),'testing')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report")
report = classification_report(y_test, test_pred, zero_division=1)
print(report)
print("--------------------------------------------------------")

## Over Sampled Scaled Data with Untuned RandomForestClassifier

In [ ]:
# Import RandomOverSampler from imblearn
from imblearn.over_sampling import RandomOverSampler

# Instantiate a RandomOversampler instance
ros = RandomOverSampler(random_state=1)

# Fit the training data to the `RandomOverSampler` model
X_oversampled, y_oversampled = ros.fit_resample(X_train_scaled, y_train)

# Count distinct values
y_oversampled.value_counts()

In [ ]:
# Instantiate the model
ros_clf = RandomForestClassifier(random_state=1)

# Fit the model
ros_clf.fit(X_oversampled, y_oversampled)

# Predict using the model
train_pred = ros_clf.predict(X_train_scaled)
test_pred = ros_clf.predict(X_test_scaled)

# Generate classification report
# Print scores
print("Balanced Accuracy Scores")
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training')
print(balanced_accuracy_score(y_test, test_pred),'testing')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report")
report = classification_report(y_test, test_pred, zero_division=1)
print(report)
print("--------------------------------------------------------")

## Under Sampled Scaled Data with Untuned RandomForestClassifier

In [ ]:
# Import RandomUnderSampler from imblearn
from imblearn.under_sampling import RandomUnderSampler

# Instantiate a RandomOversampler instance
rus = RandomUnderSampler(random_state=1)

# Fit the training data to the `RandomOverSampler` model
X_undersampled, y_undersampled = rus.fit_resample(X_train_scaled, y_train)

# Count distinct values
y_undersampled.value_counts()

In [ ]:
# Instantiate the model
rus_clf = RandomForestClassifier(random_state=1)

# Fit the model
rus_clf.fit(X_undersampled, y_undersampled)

# Predict using the model
train_pred = rus_clf.predict(X_train_scaled)
test_pred = rus_clf.predict(X_test_scaled)

# Generate classification report
# Print scores
print("Balanced Accuracy Scores")
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training')
print(balanced_accuracy_score(y_test, test_pred),'testing')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report")
report = classification_report(y_test, test_pred, zero_division=1)
print(report)
print("--------------------------------------------------------")

## Original Scaled Data and Untuned XGBoost Classifier

In [ ]:
# Define the model
xgb_clf = XGBClassifier(random_state=1)

# Fit the model
xgb_clf.fit(X_train_scaled, y_train)

# Predict using the model
train_pred = xgb_clf.predict(X_train_scaled)
test_pred = xgb_clf.predict(X_test_scaled)

# Plot feature importance
from xgboost import plot_importance
plot_importance(xgb_clf)
plt.show()

# Generate classification report
# Print scores
print("Balanced Accuracy Scores")
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training')
print(balanced_accuracy_score(y_test, test_pred),'testing')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report")
report = classification_report(y_test, test_pred, zero_division=1)
print(report)
print("--------------------------------------------------------")

## Under Sampled Scaled Data and Untuned XGBoost Classifier

In [ ]:
# Define the model
xgb_rus_clf = XGBClassifier(random_state=1)

# Fit the model
xgb_rus_clf.fit(X_undersampled, y_undersampled)

# Predict using the model
train_pred = xgb_rus_clf.predict(X_train_scaled)
test_pred = xgb_rus_clf.predict(X_test_scaled)

# Plot feature importance
plot_importance(xgb_rus_clf)
plt.show()

# Generate classification report
# Print scores
print("Balanced Accuracy Scores")
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training')
print(balanced_accuracy_score(y_test, test_pred),'testing')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report")
report = classification_report(y_test, test_pred, zero_division=1)
print(report)
print("--------------------------------------------------------")

## Under Sampled Scaled Data with XGBoost Parameter Tuning

In [ ]:
# Try the following values for learning_rate
rate = np.arange(0.01, 1.1, 0.01)
models = {'train_score': [], 'test_score': [], 'learning_rate': []}

# Loop through each value in learning_rates
for r in rate:
    # Initialize the classifier with parameter variables
    xgb_rus_clf = XGBClassifier(learning_rate = r, tree_method='approx', random_state=1)

    # Fit the undersampled data the new model
    xgb_rus_clf.fit(X_undersampled, y_undersampled)

    # Make predictions
    train_pred = xgb_rus_clf.predict(X_train_scaled)
    test_pred = xgb_rus_clf.predict(X_test_scaled)

    # Calculate balanced accuracy scores
    train_score = balanced_accuracy_score(y_train, train_pred)
    test_score = balanced_accuracy_score(y_test, test_pred)

    # Append scores
    models['train_score'].append(train_score)
    models['test_score'].append(test_score)
    models['learning_rate'].append(r)

# Create a dataframe from the models dictionary with learning_rate as the index
models_df = pd.DataFrame(models).set_index('learning_rate')

# Display df
display(models_df.sort_values(by='test_score', ascending=False).head(3))

In [ ]:
# Sort the dataframe by test_score in descending order and get the top 3
top_learning_rates = models_df.sort_values(by='test_score', ascending=False).head(3).index.tolist()

# Display the top 3 
top_learning_rates

In [ ]:
# Plot the results
models_df.plot()

In [ ]:
# Try the following values for n_estimators
estimators = range(100, 200, 1)
models = {'train_score': [], 'test_score': [], 'n_estimators': []}

# Loop through each value in n_estimators
for n in estimators:
    # Initialize the classifier with parameter variables
    xgb_rus_clf = XGBClassifier(n_estimators = n, tree_method='approx', random_state=1)

    # Fit the undersampled data the new model
    xgb_rus_clf.fit(X_undersampled, y_undersampled)

    # Make predictions
    train_pred = xgb_rus_clf.predict(X_train_scaled)
    test_pred = xgb_rus_clf.predict(X_test_scaled)

    # Calculate balanced accuracy scores
    train_score = balanced_accuracy_score(y_train, train_pred)
    test_score = balanced_accuracy_score(y_test, test_pred)

    # Append scores
    models['train_score'].append(train_score)
    models['test_score'].append(test_score)
    models['n_estimators'].append(n)

# Create a dataframe from the models dictionary with n_estimators as the index
models_df = pd.DataFrame(models).set_index('n_estimators')

# Display df
display(models_df.sort_values(by='test_score', ascending=False).head(3))

In [ ]:
# Sort the dataframe by test_score in descending order and get the top 3
top_n_estimators = models_df.sort_values(by='test_score', ascending=False).head(3).index.tolist()

# Display the top 3
top_n_estimators

In [ ]:
# Plot the results
models_df.plot()

In [ ]:
# Try the following values for max_depth
max_depths = range(1, 30)
models = {'train_score': [], 'test_score': [], 'max_depth': []}

# Loop through each value in max_depths
for depth in max_depths:
    # Initialize the classifier with parameter variables
    xgb_rus_clf = XGBClassifier(max_depth = depth, tree_method='approx', random_state=1)
    
    # Fit the undersampled data the new model
    xgb_rus_clf.fit(X_undersampled, y_undersampled)

    # Make predictions
    train_pred = xgb_rus_clf.predict(X_train_scaled)
    test_pred = xgb_rus_clf.predict(X_test_scaled)

    # Caclulate balanced accuracy scores
    train_score = balanced_accuracy_score(y_train, train_pred)
    test_score = balanced_accuracy_score(y_test, test_pred)

    # Append scores
    models['train_score'].append(train_score)
    models['test_score'].append(test_score)
    models['max_depth'].append(depth)

# Create a dataframe from the models dictionary with max_depth as the index
models_df = pd.DataFrame(models).set_index('max_depth')

# Display df
display(models_df.sort_values(by='test_score', ascending=False).head(3))

In [ ]:
# Sort the dataframe by test_score in descending order and get the top 3
top_max_depth = models_df.sort_values(by='test_score', ascending=False).head(3).index.tolist()

# Display the top 3
top_max_depth

In [ ]:
# Plot the results
models_df.plot()

In [ ]:
# Try the following values for max_leaves
max_leaves = range(1, 30)
models = {'train_score': [], 'test_score': [], 'max_leaves': []}

# Loop through each value in max_depths
for leaf in max_leaves:
    # Initialize the classifier with parameter variables
    xgb_rus_clf = XGBClassifier(max_leaves = leaf, tree_method='approx', random_state=1)
    
    # Fit the undersampled data the new model
    xgb_rus_clf.fit(X_undersampled, y_undersampled)

    # Make predictions
    train_pred = xgb_rus_clf.predict(X_train_scaled)
    test_pred = xgb_rus_clf.predict(X_test_scaled)

    # Caclulate balanced accuracy scores
    train_score = balanced_accuracy_score(y_train, train_pred)
    test_score = balanced_accuracy_score(y_test, test_pred)

    # Append scores
    models['train_score'].append(train_score)
    models['test_score'].append(test_score)
    models['max_leaves'].append(leaf)

# Create a dataframe from the models dictionary with max_depth as the index
models_df = pd.DataFrame(models).set_index('max_leaves')

# Display df
display(models_df.sort_values(by='test_score', ascending=False).head(3))

In [ ]:
# Sort the dataframe by test_score in descending order and get the top 3
top_max_leaves = models_df.sort_values(by='test_score', ascending=False).head(3).index.tolist()

# Display the top 3
top_max_leaves

In [ ]:
# Plot the results
models_df.plot()

In [ ]:
# Try the following values for subsample
subsample = np.arange(0, 1, .01)
models = {'train_score': [], 'test_score': [], 'subsample': []}

# Loop through each value in max_depths
for sample in subsample:
    # Initialize the classifier with parameter variables
    xgb_rus_clf = XGBClassifier(subsample = sample, tree_method='approx', random_state=1)
    
    # Fit the undersampled data the new model
    xgb_rus_clf.fit(X_undersampled, y_undersampled)

    # Make predictions
    train_pred = xgb_rus_clf.predict(X_train_scaled)
    test_pred = xgb_rus_clf.predict(X_test_scaled)

    # Caclulate balanced accuracy scores
    train_score = balanced_accuracy_score(y_train, train_pred)
    test_score = balanced_accuracy_score(y_test, test_pred)

    # Append scores
    models['train_score'].append(train_score)
    models['test_score'].append(test_score)
    models['subsample'].append(sample)

# Create a dataframe from the models dictionary with max_depth as the index
models_df = pd.DataFrame(models).set_index('subsample')

# Display df
display(models_df.sort_values(by='test_score', ascending=False).head(3))

In [ ]:
# Sort the dataframe by test_score in descending order and get the top 3
top_subsample = models_df.sort_values(by='test_score', ascending=False).head(3).index.tolist()

# Display the top 3
top_subsample

In [ ]:
# Plot the results
models_df.plot()

In [ ]:
# Dictionary with 3 values for each parameter
params = {
    'n_estimators': top_n_estimators,
    'max_depth': top_max_depth,
    'learning_rate': top_learning_rates,
    'max_leaves': top_max_leaves,
    'subsample': top_subsample
}

# Initialize a dictionary to store results
results = {'n_estimators': [], 'max_depth': [], 'learning_rate': [],'max_leaves': [], 'subsample': [], 'train_score': [], 'test_score': []}

# Loop through all combinations of parameters
for n in params['n_estimators']:
    for d in params['max_depth']:
        for lr in params['learning_rate']:
            for ml in params['max_leaves']:
                for s in params['subsample']:
                    # Initialize the classifier with current parameters
                    xgb_rus_clf = XGBClassifier(n_estimators=n, learning_rate=lr, max_depth=d, max_leaves=ml, subsample=s, tree_method='approx', random_state=1)
                
                    # Fit the undersampled data
                    xgb_rus_clf.fit(X_undersampled, y_undersampled)
    
                    # Make predictions
                    train_pred = xgb_rus_clf.predict(X_train_scaled)
                    test_pred = xgb_rus_clf.predict(X_test_scaled)
    
                    # Calculate balanced accuracy scores
                    train_score = balanced_accuracy_score(y_train, train_pred)
                    test_score = balanced_accuracy_score(y_test, test_pred)
    
                    # Append results to the dictionary
                    results['n_estimators'].append(n)
                    results['max_depth'].append(d)
                    results['learning_rate'].append(lr)
                    results['max_leaves'].append(ml)
                    results['subsample'].append(s)
                    results['train_score'].append(train_score)
                    results['test_score'].append(test_score)

# Create a DataFrame from the results dictionary
results_df = pd.DataFrame(results)

In [ ]:
sorted_results_df = results_df.sort_values(by='test_score', ascending=False)
print(sorted_results_df.head(3))

In [ ]:
# For Classification
# Initialize the classifier with current parameters
xgb_rus_clf = XGBClassifier(n_estimators=162, max_depth=15, learning_rate=.02, max_leaves=29, subsample=.43, tree_method='approx', min_split_loss=0, min_child_weight=1, random_state=1)

# Fit the undersampled data
xgb_rus_clf.fit(X_undersampled, y_undersampled)

# Make predictions
train_pred = xgb_rus_clf.predict(X_train_scaled)
test_pred = xgb_rus_clf.predict(X_test_scaled)

# Generate classification report
# Print scores
print("Balanced Accuracy Scores")
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training score')
print(balanced_accuracy_score(y_test, test_pred),'testing score')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report - Original Data")
print(classification_report(y_test, test_pred))
print("--------------------------------------------------------")

## Original Scaled Data and Untuned LightBoost

In [ ]:
# Define model
lgb_clf = lgb.LGBMClassifier(random_state=1, verbose=-1)

# Fit the model
lgb_clf.fit(X_train_scaled, y_train)

# Predict using the model
train_pred = lgb_clf.predict(X_train_scaled)
test_pred = lgb_clf.predict(X_test_scaled)

# Generate classification report
# Print scores
print("Balanced Accuracy Scores")
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training')
print(balanced_accuracy_score(y_test, test_pred),'testing')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report")
report = classification_report(y_test, test_pred, zero_division=1)
print(report)
print("--------------------------------------------------------")

## Under Sampled Scaled Data with Untuned LightBoost

In [ ]:
# Define model
lgb_rus_clf = lgb.LGBMClassifier(random_state=1, verbose=-1)

# Fit the model
lgb_rus_clf.fit(X_undersampled, y_undersampled)

# Predict using the model
train_pred = lgb_rus_clf.predict(X_train_scaled)
test_pred = lgb_rus_clf.predict(X_test_scaled)

# Generate classification report
# Print scores
print("Balanced Accuracy Scores")
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training')
print(balanced_accuracy_score(y_test, test_pred),'testing')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report")
report = classification_report(y_test, test_pred, zero_division=1)
print(report)
print("--------------------------------------------------------")

## RandomUnderSampler Data with Tuned Lightboost Classifier

In [ ]:
rates = np.arange(0.05, 1.0, 0.01)
models = {'train_score': [], 'test_score': [], 'learning_rate': []}

# Loop through each value in rates
for r in rates:
    # Initialize the classifier with current parameters
    lgb_rus_clf = lgb.LGBMClassifier(learning_rate=r, random_state=1, verbose=-1)

    # Fit the undersampled data
    lgb_rus_clf.fit(X_undersampled, y_undersampled)

    # Make predictions
    train_pred = lgb_rus_clf.predict(X_train)
    test_pred = lgb_rus_clf.predict(X_test)

    # Calculate balanced accuracy score
    train_score = balanced_accuracy_score(y_train, train_pred)
    test_score = balanced_accuracy_score(y_test, test_pred)

    # Append scores
    models['train_score'].append(train_score)
    models['test_score'].append(test_score)
    models['learning_rate'].append(r)

# Create a dataframe from the models dictionary with learning_rate as the index
models_df = pd.DataFrame(models).set_index('learning_rate')

# Display df
display(models_df.sort_values(by='test_score', ascending=False).head(3))

In [ ]:
rates = np.arange(0.05, 1.0, 0.01)
models = {'train_score': [], 'test_score': [], 'learning_rate': []}

# Loop through each value in rates
for r in rates:
    # Initialize the classifier with current parameters
    lgb_rus_clf = lgb.LGBMClassifier(learning_rate=r, random_state=1, verbose=-1)

    # Fit the undersampled data
    lgb_rus_clf.fit(X_undersampled, y_undersampled)

    # Make predictions
    train_pred = lgb_rus_clf.predict(X_train)
    test_pred = lgb_rus_clf.predict(X_test)

    # Calculate balanced accuracy score
    train_score = balanced_accuracy_score(y_train, train_pred)
    test_score = balanced_accuracy_score(y_test, test_pred)

    # Append scores
    models['train_score'].append(train_score)
    models['test_score'].append(test_score)
    models['learning_rate'].append(r)

# Create a dataframe from the models dictionary with learning_rate as the index
models_df = pd.DataFrame(models).set_index('learning_rate')

# Display df
display(models_df.sort_values(by='test_score', ascending=False).head(3))

In [ ]:
# Plot the results
models_df.plot()

In [ ]:
# Try the following values for n_estimators
estimators = range(100, 250, 1)
models = {'train_score': [], 'test_score': [], 'n_estimators': []}

# Loop through each value in n_estimators
for n in estimators:
    # Initialize the classifier with current parameters
    lgb_rus_clf =lgb.LGBMClassifier(n_estimators = n, random_state=1, verbose=-1)

    # Fit the undersampled data
    lgb_rus_clf.fit(X_undersampled, y_undersampled)

    # Make predictions
    train_pred = lgb_rus_clf.predict(X_train)
    test_pred = lgb_rus_clf.predict(X_test)

    # Calculate balanced accuracy scores
    train_score = balanced_accuracy_score(y_train, train_pred)
    test_score = balanced_accuracy_score(y_test, test_pred)

    # Append scores
    models['train_score'].append(train_score)
    models['test_score'].append(test_score)
    models['n_estimators'].append(n)

# Create a dataframe from the models dictionary with n_estimators as the index
models_df = pd.DataFrame(models).set_index('n_estimators')

# Display df
display(models_df.sort_values(by='test_score', ascending=False).head(3))

In [ ]:
# Sort the dataframe by test_score in descending order and get the top 3
n_estimators = models_df.sort_values(by='test_score', ascending=False).head(3).index.tolist()

# Display the top 3
n_estimators

In [ ]:
# Plot the results
models_df.plot()

In [ ]:
# Try the following values for max_depth
max_depths = range(2, 30)
models = {'train_score': [], 'test_score': [], 'max_depth': []}

# Loop through each value in max_depths
for depth in max_depths:
    # Initialize the classifier with current parameters
    lgb_rus_clf = lgb.LGBMClassifier(max_depth = depth, random_state=1, verbose=-1)
    
    # Fit the undersampled data
    lgb_rus_clf.fit(X_undersampled, y_undersampled)

    # Make predictions
    train_pred = lgb_rus_clf.predict(X_train)
    test_pred = lgb_rus_clf.predict(X_test)

    # Calculate balanced accuracy scores
    train_score = balanced_accuracy_score(y_train, train_pred)
    test_score = balanced_accuracy_score(y_test, test_pred)
 
    # Append scores
    models['train_score'].append(train_score)
    models['test_score'].append(test_score)
    models['max_depth'].append(depth)

# Create a dataframe from the models dictionary with max_depth as the index
models_df = pd.DataFrame(models).set_index('max_depth')

# Display df
display(models_df.sort_values(by='test_score', ascending=False).head(3))

In [ ]:
# Sort the dataframe by test_score in descending order and get the top 3
max_depth = models_df.sort_values(by='test_score', ascending=False).head(3).index.tolist()

# Display the top 3 
max_depth

In [ ]:
# Plot the results
models_df.plot()

In [ ]:
# Try the following values for num_leaves
num_leaves = range(10, 50)
models = {'train_score': [], 'test_score': [], 'num_leaves': []}

# Loop through each value in num_leaves
for num in num_leaves:
    # Initialize the classifier with current parameters
    lgb_rus_clf = lgb.LGBMClassifier(num_leaves = num, random_state=1, verbose=-1)
    
    # Fit the undersampled data
    lgb_rus_clf.fit(X_undersampled, y_undersampled)

    # Make predictions
    train_pred = lgb_rus_clf.predict(X_train)
    test_pred = lgb_rus_clf.predict(X_test)

    # Calculate balanced accuracy scores
    train_score = balanced_accuracy_score(y_train, train_pred)
    test_score = balanced_accuracy_score(y_test, test_pred)
 
    # Append scores
    models['train_score'].append(train_score)
    models['test_score'].append(test_score)
    models['num_leaves'].append(num)

# Create a dataframe from the models dictionary with max_depth as the index
models_df = pd.DataFrame(models).set_index('num_leaves')

# Display df
display(models_df.sort_values(by='test_score', ascending=False).head(3))

In [ ]:
# Sort the dataframe by test_score in descending order and get the top 3
num_leaves = models_df.sort_values(by='test_score', ascending=False).head(3).index.tolist()

# Display the top 3 
num_leaves

In [ ]:
# Plot the results
models_df.plot()

In [ ]:
# Try the following values for max_bin
max_bin = range(20, 512, 1)
models = {'train_score': [], 'test_score': [], 'max_bin': []}

# Loop through each value in max_bins
for bin in max_bin:
    # Initialize the classifier with current parameters
    lgb_rus_clf = lgb.LGBMClassifier(max_bin = bin, random_state=1, verbose=-1)
    
    # Fit the undersampled data
    lgb_rus_clf.fit(X_undersampled, y_undersampled)

    # Make predictions
    train_pred = lgb_rus_clf.predict(X_train)
    test_pred = lgb_rus_clf.predict(X_test)

    # Calculate balanced accuracy scores
    train_score = balanced_accuracy_score(y_train, train_pred)
    test_score = balanced_accuracy_score(y_test, test_pred)
 
    # Append scores
    models['train_score'].append(train_score)
    models['test_score'].append(test_score)
    models['max_bin'].append(bin)

# Create a dataframe from the models dictionary with max_depth as the index
models_df = pd.DataFrame(models).set_index('max_bin')

# Display df
display(models_df.sort_values(by='test_score', ascending=False).head(3))

In [ ]:
# Sort the dataframe by test_score in descending order and get the top 3
max_bin = models_df.sort_values(by='test_score', ascending=False).head(3).index.tolist()

# Display the top 3 
max_bin

In [ ]:
# Plot the results
models_df.plot()

In [ ]:
# Dictionary with 3 values for each parameter
params = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'learning_rate': learning_rate,
    'num_leaves': num_leaves,
    'max_bin': max_bin
}

# Initialize a dictionary to store results
results = {'n_estimators': [], 'max_depth': [], 'learning_rate': [], 'num_leaves': [], 'max_bin': [], 'train_score': [], 'test_score': []}

# Loop through all combinations of parameters
for n in params['n_estimators']:
    for d in params['max_depth']:
        for lr in params['learning_rate']:
            for nl in params['num_leaves']:
                for b in params['max_bin']:
                    # Initialize the classifier with current parameters
                    lgb_rus_clf = lgb.LGBMClassifier(n_estimators=n, learning_rate=lr, max_depth=d, num_leaves=nl, max_bin=b, random_state=1, verbose=-1)
                
                    # Fit the undersampled data
                    lgb_rus_clf.fit(X_undersampled, y_undersampled)
    
                    # Make predictions
                    train_pred = lgb_rus_clf.predict(X_train)
                    test_pred = lgb_rus_clf.predict(X_test)
    
                    # Calculate balanced accuracy scores
                    train_score = balanced_accuracy_score(y_train, train_pred)
                    test_score = balanced_accuracy_score(y_test, test_pred)
    
                    # Append results to the dictionary
                    results['n_estimators'].append(n)
                    results['max_depth'].append(d)
                    results['learning_rate'].append(lr)
                    results['num_leaves'].append(nl)
                    results['max_bin'].append(b)
                    results['train_score'].append(train_score)
                    results['test_score'].append(test_score)
                    
# Create a DataFrame from the results dictionary
results_df = pd.DataFrame(results)

In [ ]:
sorted_results_df = results_df.sort_values(by='test_score', ascending=False)
print(sorted_results_df.head(3))

In [ ]:
# For Classification
# Initialize the classifier with current parameters
lgb_rus_clf = lgb.LGBMClassifier(n_estimators=105, max_depth=16, learning_rate=.06, num_leaves=22, max_bin=67, random_state=1, verbose=-1)

# Fit the undersampled data
lgb_rus_clf.fit(X_undersampled, y_undersampled)

In [ ]:
# Make predictions
train_pred = lgb_rus_clf.predict(X_train)
test_pred = lgb_rus_clf.predict(X_test)

In [ ]:
# Check the model's balanced accuracy on the training and test sets
print('\nRandomUnderSampled Data and Tuned LGB Classifer')

# Print scores
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training score')
print(balanced_accuracy_score(y_test, test_pred),'testing score')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report - Original Data")
print(classification_report(y_test, test_pred))
print("--------------------------------------------------------")

In [ ]:
# Get the feature importance array
importances = lgb_rus_clf.feature_importances_
# List the top 10 most important features
importances_sorted = sorted(zip(lgb_rus_clf.feature_importances_, X.columns), reverse=True)
importances_sorted